In [8]:
import tensorflow as tf

In [2]:
# A variable describes the weights connecting neurons between two layers of a feedforward neural network
# In this case, weights is meant to be trainable, we will automatically compute and apply gradients to weights.
weights = tf.Variable(tf.random_normal([300, 200], stddev=0.5), name="weights")

In [3]:
# If weights is not meant to be trainable, we may pass an optional flag when we call tf.Variable
weigths1 = tf.Variable(tf.random_normal([300, 200], stddev=0.5), name="weights1", trainable=False)

In [5]:
# In addition to tf.random_normal, there are several other methods to initialize a TensorFlow variable:
# Common tensors from the Tensorflow API docs
shape = [2,3]
x1 = tf.zeros(shape, dtype=tf.float32, name="x1")
x2 = tf.ones(shape, dtype=tf.float32, name="x2")
x3 = tf.random_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name="x3")
x4 = tf.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name="x4")
x5 = tf.random_uniform(shape, minval=0, maxval=100, dtype=tf.float32, seed=None , name="x5")

TensorFlow variables have the folling three properties:
- Variables must be *explicitly* initialized before a graph is used for the first time
- We can use gradient methods to modify variables after each iteration as we search for a model's optimal parameter settings
- We can save the values stored in variables to disk and restore them for later use

When we call `tf.Variable`, the following **three operations are added** to the computation graph:
- The operation producing the tensor we use to initialize our variable
- The `tf.assign` operation, which is responsible for filling the variable with the initializing tensor prior to the variable's use (All such assign ops in the graph are triggered by running `tf.initialize_all_variables()`)
- The variable operation, which holds the current value of the variable

**TensorFlow Operations**

On a high-level, TensorFlow *operations* represent abstract transformations that are applied to tensors in the computation graph. Operations may have attributes that may be supplied a priori or are inferred at runtime. For example, an attribute may serve to describe the expected types of the input (adding tensors of type float32 vs. int32). Just as variables are named, operations may also be supplied with an optional name attribute for easy reference into the computation graph.

An operation consists of one or more *kernels*, which represent device-specific implementations. For example, an operation may have separate CPU and GPU kernels because it can be more efficiently expressed on a GPU. This is the case for many TensorFlow operations on matrices.

** Placeholder Tensors **
How we pass the input to our deep model during both train and test time. A variable is insufficient because it is only meant to be initialized once. We instead need a component that we populate every single time the computation graph is run.

In [6]:
x = tf.placeholder(tf.float32, name="x", shape=[None, 784]) # a mini-batch of data stored as float32
W = tf.Variable(tf.random_uniform([784, 10], -1, 1), name="W")
multiply = tf.matmul(x, W)

While we could instead multiply each data sample separately by *W*, expressing a full mini-batch as a tensor allows us to compute the results for all the data samples in parallel.

Just as variables need to be initialized the first time the computation graph is built, placeholders need to be filled every time the computation graph (or a subgraph) is run. 

** Sessions in TensorFlow **

A TensorFlow program interacts with a computation graph using a *session*. The TensorFlow session is responsible for building the initial graph, can be used to initialize all variables appropriately, and to run the computational graph.

We then initialize the variables as required by using the session variable to run the initialization operation in *session.run(init_op)*.

We will explore two more major concepts in building and maintaining computational graphs.
** Variable Scopes ** and ** Sharing Variables **
Building complex models often requires re-using and sharing large sets of variables that we'll want to instantiate together in one place.
In many cases, we don't want to create a copy, but instead, we want to reuse the model and its variables. It turns out, in this case, we shouldn't be using `tf.Variable`. Instead, we should using a more advanced naming scheme that takes advantage of TensorFlow's variable scoping.

TensorFlow's variable scoping mechanisms are largely controlled by two functions:
- *tf.get_variable(name,shape,initializer)*: checks if a variable with this name exists, retrieves the variable if it does, creates it using the shape and initializer if it doesn't.
- *tf.variable_scope(scope_name)*: manages the namespace and determines the scope in which *tf.get_variable* operates

Unlike *tf.Variable*, the *tf.get_variable* command checks that a variable of the given name hasn't already been instantiated. By default, sharing is not allowed (just to be safe!), but if you want to enable sharing within a variable scope, we can say so explictly:

Enable *sharing* within a variable scope:
- with tf.variable_scope("shared_variables") as scope:
    ...
    scope.reuse_variables()

** Managing Models over the CPU and GPU **

TensorFlow allows us to utilize multiple computing devices if we so desire to build and train our models. Supported devices are represented by string ID's and normally consist of the following:
- "/cpu:0": The cpu of our machine.
- "/gpu:0": The first gpu of our machine, if it has one.
- "/gpu:1": The second gpu of our machine, if it has one.

To inspect which devices are used by the computational graph, we can initialize our TensorFlow session with the *log_device_placement* set to *True*:

In [9]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [10]:
with tf.device('/gpu:2'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0], shape=[2,2], name='a')
    b = tf.constant([1.0, 2.0], shape=[2,1], name='b')
    c = tf.matmul(a, b)
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
sess.run(c)

array([[  5.],
       [ 11.]], dtype=float32)

In [ ]:
c = []
